Version Control:

Sept 4, 2018
V.0  - Copy from "Generate Stock Pattern JPG v.0.ipynb".  
V.1  - Change TrainSet period with 4 months instead of 1 year.
V.1a - Update documentation and change path by using variable.

# Step 1. Generate stock pattern to jpg

## Four parameters to be set for jpg generation:

TrainSet Data = C:\..\1.Project\0.Stock\1.Data\Train_4M\TrainSet_Label\[year]-[period]-[trainset]
TrainSet Log = C:\..\1.Project\01.Stock\1.Data\Train_4M\Source\Log

1) Set Year 
2) Set Period:
    1 = Jan 1 - Apr 30
    2 = May 1 - Aug 31
    3 = Sep 1 - Dec 31
3) Set Train Set: A / B / C
4) Set samplefile: 
        "Y" for sample file
        "N" for real stock batch
5) Set train_data_path (under this path, please create empty folder e.g. 2018-1-A to store the jpg)
6) Set train_log_path 

In [1]:
#  Please update the following three parameters

year = 2018
period = 2
trainset_no = "B"

samplefile = "N" 

train_data_path = "C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/"
train_log_path = "C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/Log/"
train_label_path = 'C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/'

trainset = str(year) + "-" + str(period) + "-" + trainset_no

In [2]:
# Fix Yahoo API issue: https://github.com/pydata/pandas-datareader/issues/545

import pandas as pd
pd.core.common.is_list_like = pd.api.types.is_list_like
from pandas_datareader import data, wb
import fix_yahoo_finance as yf
yf.pdr_override()
import numpy as np
from datetime import datetime
# import seaborn as sns
import matplotlib.pyplot as plt

In [10]:
# Automatically set start & end period and read-in StockCode file

if period == 1 :
    start = datetime(year, 1, 1)
    end = datetime(year, 4, 30)
elif period == 2 :
    start = datetime(year, 5, 1)
    end = datetime(year, 8, 31)
else:
    start = datetime(year, 9, 1)
    end = datetime(year, 12, 31)
        
# Get stockcode file 
if samplefile == "Y":
#     stockcodes = "C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/Train_Sample.csv"
    stockcodes = train_data_path + "Train_Sample.csv"
else:
#     stockcodes = "C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/" + trainset_no + ".csv"
    stockcodes = train_data_path + trainset_no + ".csv"

df_batch = pd.read_csv(stockcodes)

# print("Start Time: ", datetime.today().now())


In [13]:
#  Generate JPG stock pattern files to respective folder

# f = open('Train_4M/Log/' + str(year) + '_' + str(period) + "-" + trainset_no + '.log', 'a')
f = open(train_log_path + str(year) + '_' + str(period) + "-" + trainset_no + '.log', 'a')

f.write('\nRun Time: %s' % datetime.today().now())
f.close()  

batch_Train = df_batch.iloc[:,0].tolist()
for batch in batch_Train:
        batch_fn = batch.rstrip(".HK")
        
        try: 
            ticker = data.get_data_yahoo(batch, start, end)
            df = pd.DataFrame(index=ticker.index)
            df[batch] = ticker["Close"]
            plt.plot(df[batch], label = batch)
            plt.axis("off")
##           plt.savefig("TrainSet/TrainSet_2017_Train1/2017-" + batch_fn +"-U.jpg")
# plt.savefig("Train_4M/" + str(year) + "-" + str(period) + "-" + trainset_no + "/" + str(year) + "-" + batch_fn +"-U.jpg")
            plt.savefig(train_data_path + str(year) + "-" + str(period) + "-" + trainset_no + "/" + str(year) + "-" + batch_fn +"-U.jpg")
            plt.close()
        except :
##           f = open('C:/Users/Sonia/1.Project/0.Stock/TrainSet/Log/2017_Train1.log', 'a')
#    f = open('Train_4M/Log/' + str(year) + '_' + str(period) + "-" + trainset_no + '.log', 'a')
            f = open(train_log_path + str(year) + '_' + str(period) + "-" + trainset_no + '.log', 'a')
            f.write('\nAn exceptional thing happened - %s' % batch)
            f.close()   
            pass           

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded


# Step 2. Rename jpg filename with respective folders

## Two parameters to be updated:
1) trainsetpath
2) trainset

-U Up (No required to change as all files are generated with -U by default)
-D Down (Required to rename)
-N Neutral (Required to rename)

In [3]:
# Rename file at Windows level
# Set two parameters:
# 1. Move the lablled files to trainsetpath
# 2. Set trainset 

import os 

# train_label_path = 'C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/'
# trainset = '2018-1-A'

filefullpath =[]
# folder = 'C:/Users/Sonia/1.Project/0.Stock/TrainSet_Label/TrainSet_2015_Train1/Down/'
folder = train_label_path + trainset + "/Down/"

# for i in os.listdir('C:/Users/Sonia/1.Project/0.Stock/TrainSet_Label/TrainSet_2015_Train1/.'):
for i in os.listdir(folder+'.'):
    filefullpath = (folder + i)
    [os.rename(filefullpath, filefullpath.replace('-U', '-D'))]
    
# folder = 'C:/Users/Sonia/1.Project/0.Stock/TrainSet_Label/TrainSet_2015_Train1/Neutral/'
folder = train_label_path + trainset + "/Neutral/"
# for i in os.listdir('C:/Users/Sonia/1.Project/0.Stock/TrainSet_Label/TrainSet_2015_Train1/.'):
for i in os.listdir(folder+'.'):
    filefullpath = (folder + i)
    [os.rename(filefullpath, filefullpath.replace('-U', '-N'))]
    

# Step 3. Read in TrainSet and label with 1 (up), 0 (neutral), -1(down)

In [4]:
import csv
import os

stocklabel = []
stocklabel1 = ()

# trainsetpath = 'C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/'
# trainset = '2018-1-A'

# Label Down folder
lists = []
# for file in os.listdir('C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/2018-1-A/Down/.' ):
for file in os.listdir(train_label_path + trainset + "/Down/."):
    lists.append(file)
for i in range(len(lists)): 
    stocklabel1 = (lists[i], '-1')
    stocklabel.append(stocklabel1)
    
# Label Up folder
lists = []
# for file in os.listdir('C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/2018-1-A/Up/.'):
for file in os.listdir(train_label_path + trainset + "/Up/."):
    lists.append(file)
for i in range(len(lists)): 
    stocklabel1 = (lists[i], '1')
    stocklabel.append(stocklabel1)

# Label Neutral folder
lists = []
# for file in os.listdir('C:/Users/Sonia/1.Project/01.Stock/1.Data/Train_4M/TrainSet_Label/2018-1-A/Neutral/.'):
for file in os.listdir(train_label_path + trainset + "/Neutral/."):
    lists.append(file)
for i in range(len(lists)): 
    stocklabel1 = (lists[i], '0')
    stocklabel.append(stocklabel1)


In [5]:
len(stocklabel)


551

# Step 4. Get stock data from Yahoo and write to result to JSON format

In [6]:
# i = ('2015-0004-D.jpg', '-1')

d = {}
tickers = []

for i in stocklabel:
    k, v = i
    d[k] = v
    
# d.keys()
# d.values()
list01 = [f"{i[5:9]}.HK" for i in d.keys()]
list02 = [i for i in d.values()]
tickers = list(zip(list01,list02))


In [7]:
len(tickers)

551

In [8]:
# Aug 20, 2018 - change to write result file to JSON format

from datetime import datetime

if period == 1 :
    start = datetime(year, 1, 1)
    end = datetime(year, 4, 30)
elif period == 2 :
    start = datetime(year, 5, 1)
    end = datetime(year, 8, 31)
else:
    start = datetime(year, 9, 1)
    end = datetime(year, 12, 31)
    
result = {}


for i,j in tickers:
    try:
#       print(i,j)
        ticker = data.get_data_yahoo(i,start,end)
        Adj_Close = list(ticker["Adj Close"])
        result[i] = {'data':Adj_Close,'label':j}
#         print(result.keys())
    except:
        f = open(train_label_path + "Log/Log.log", 'a')
        f.write('\nAn exceptional thing happened - %s' %i)
        f.close()   
        pass 


[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*********************100%***********************]  1 of 1 downloaded
[*******************

In [9]:
import json

with open(train_label_path + trainset + "_result.text",'w') as f:
    json.dump(result,f)

In [8]:
result

{'1456.HK': {'data': [3.76,
   3.87,
   3.87,
   3.88,
   3.72,
   3.77,
   3.72,
   3.65,
   3.71,
   3.75,
   3.79,
   3.82,
   3.81,
   3.8,
   3.89,
   3.87,
   3.88,
   3.87,
   3.88,
   3.85,
   3.79,
   3.7,
   3.62,
   3.6,
   3.6,
   3.32,
   3.34,
   3.39,
   3.16,
   3.2,
   3.28,
   3.34,
   3.37,
   3.46,
   3.47,
   3.63,
   3.65,
   3.7,
   3.59,
   3.58,
   3.53,
   3.53,
   3.39,
   3.41,
   3.33,
   3.32,
   3.4,
   3.38,
   3.39,
   3.4,
   3.38,
   3.4,
   3.4,
   3.39,
   3.39,
   3.31,
   3.26,
   3.18,
   3.18,
   3.12,
   3.12,
   2.99,
   2.9,
   2.94,
   2.91,
   2.95,
   2.96,
   2.94,
   2.88,
   2.8,
   2.86,
   2.76,
   2.81,
   2.73,
   2.76,
   2.76,
   2.85,
   2.71,
   2.74,
   2.75],
  'label': '-1'},
 '1458.HK': {'data': [7.826778,
   7.826778,
   8.161088000000001,
   8.111925,
   8.062761,
   8.200417999999999,
   8.082427000000001,
   8.161088000000001,
   7.964436,
   7.669455999999999,
   7.75795,
   8.023431,
   7.925105,
   7.875942,
   7.7874

In [ ]:
# counting 
# 2017 Train1 - {241, 148, 228, 247}


s = set()
for i in result.keys():
        s.add(len(result[i]['data']))
print(s)  

# Step 5. Count no of days 